**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Oct 10 06:29:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 Art_Generation_with_Neural_Style_Transfer_v3a.ipynb   yolov3
'Getting started.pdf'				       yolo_v3
'google collab'					      'yolov3 all set'


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14330, done.
remote: Total 14330 (delta 0), reused 0 (delta 0), pack-reused 14330
Receiving objects: 100% (14330/14330), 12.87 MiB | 24.50 MiB/s, done.
Resolving deltas: 100% (9776/9776), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [8]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

In [7]:
!echo "Car" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [8]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-10-10 06:32:02--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   276KB/s    in 14m 2s  

2020-10-10 06:46:06 (188 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [9]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/vid_4_1000.jpg  
  inflating: data/obj/vid_4_1000.txt  
  inflating: data/obj/vid_4_10000.jpg  
  inflating: data/obj/vid_4_10000.txt  
  inflating: data/obj/vid_4_10020.jpg  
  inflating: data/obj/vid_4_10020.txt  
  inflating: data/obj/vid_4_10040.jpg  
  inflating: data/obj/vid_4_10040.txt  
  inflating: data/obj/vid_4_10060.jpg  
  inflating: data/obj/vid_4_10060.txt  
  inflating: data/obj/vid_4_10100.jpg  
  inflating: data/obj/vid_4_10100.txt  
  inflating: data/obj/vid_4_10120.jpg  
  inflating: data/obj/vid_4_10120.txt  
  inflating: data/obj/vid_4_10140.jpg  
  inflating: data/obj/vid_4_10140.txt  
  inflating: data/obj/vid_4_1020.jpg  
  inflating: data/obj/vid_4_1020.txt  
  inflating: data/obj/vid_4_1040.jpg  
  inflating: data/obj/vid_4_1040.txt  
  inflating: data/obj/vid_4_10480.jpg  
  inflating: data/obj/vid_4_10480.txt  
  inflating: data/obj/vid_4_10500.jpg  
  inflating: data/obj/vid_4_10500.txt  
  inflati

In [10]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)



[]
0 0.32561505059171597 0.5037805663157895 0.5043415340236685 0.635054697631579

['0', '0.32561505059171597', '0.5037805663157895', '0.5043415340236685', '0.635054697631579']
0 data/obj/vid_4_1980.txt
0 0.32561505059171597 0.5037805663157895 0.5043415340236685 0.635054697631579
0 0.5202604920118343 0.49991956236842106 0.6649782923076922 0.6093146718421053

['0', '0.5202604920118343', '0.49991956236842106', '0.6649782923076922', '0.6093146718421053']
0 data/obj/vid_4_1980.txt
0 0.5202604920118343 0.49991956236842106 0.6649782923076922 0.6093146718421053
0 0.8068017366863905 0.46517052763157896 0.9833574529585799 0.6015926642105263
['0', '0.8068017366863905', '0.46517052763157896', '0.9833574529585799', '0.6015926642105263']
0 data/obj/vid_4_1980.txt
0 0.8068017366863905 0.46517052763157896 0.9833574529585799 0.6015926642105263


[]
0 0.04124457307692308 0.5153635778947369 0.23444283653846154 0.6672297297368421
['0', '0.04124457307692308', '0.5153635778947369', '0.23444283653846154', 

In [11]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/vid_4_9880.jpg', 'data/obj/vid_4_6320.jpg', 'data/obj/vid_4_22240.jpg', 'data/obj/vid_4_1820.jpg', 'data/obj/vid_4_18200.jpg', 'data/obj/vid_4_9220.jpg', 'data/obj/vid_4_17100.jpg', 'data/obj/vid_4_15040.jpg', 'data/obj/vid_4_8720.jpg', 'data/obj/vid_4_26340.jpg', 'data/obj/vid_4_12160.jpg', 'data/obj/vid_4_21660.jpg', 'data/obj/vid_4_13700.jpg', 'data/obj/vid_4_12260.jpg', 'data/obj/vid_4_8680.jpg', 'data/obj/vid_4_8560.jpg', 'data/obj/vid_4_3240.jpg', 'data/obj/vid_4_3840.jpg', 'data/obj/vid_4_1860.jpg', 'data/obj/vid_4_880.jpg', 'data/obj/vid_4_16160.jpg', 'data/obj/vid_4_9740.jpg', 'data/obj/vid_4_12180.jpg', 'data/obj/vid_4_10040.jpg', 'data/obj/vid_4_9460.jpg', 'data/obj/vid_4_21520.jpg', 'data/obj/vid_4_900.jpg', 'data/obj/vid_4_11960.jpg', 'data/obj/vid_4_21400.jpg', 'data/obj/vid_4_11400.jpg', 'data/obj/vid_4_29460.jpg', 'data/obj/vid_4_11420.jpg', 'data/obj/vid_4_1960.jpg', 'data/obj/vid_4_29940.jpg', 'data/obj/vid_4_11380.jpg', 'data/obj/vid_4_12280.jpg', 'data/ob

In [12]:
len(images_list)

355

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.690282, GIOU: 0.690282), Class: 0.998740, Obj: 0.801434, No Obj: 0.003413, .5R: 1.000000, .75R: 0.400000, count: 5, class_loss = 0.282099, iou_loss = 0.220933, total_loss = 0.503032 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000006, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000003, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 105024, rewritten_bbox = 1.650099 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.801649, GIOU: 0.792188), Class: 0.996083, Obj: 0.520931, No Obj: